In [1]:
from dateutil.relativedelta import relativedelta
from datetime import datetime
from calendar import monthrange
from circle_cal.model import CalendarElement

In [2]:
c = CalendarElement(year=2024, day=1)

In [6]:
c.relativedelta.year

2024

In [4]:
dir(c)

['RELUNITS',
 'UNITRANGES',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'day',
 'hour',
 'iterover',
 'microsecond',
 'minute',
 'month',
 'relativedelta',
 'second',
 'subunits',
 'unit',
 'unitranges',
 'year']

In [8]:
dir(c)

['RELUNITS',
 'UNITRANGES',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'day',
 'hour',
 'iterover',
 'microsecond',
 'minute',
 'month',
 'relativedelta',
 'second',
 'subunits',
 'unit',
 'unitranges',
 'year']

In [9]:
c.day

In [10]:
c.year